In [1]:
!pip install kagglehub

In [2]:
import kagglehub
import os
import shutil
import glob
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img, array_to_img

# Step 1 - Download the dataset
path = kagglehub.dataset_download("madhushreesannigrahi/fish-recognition-ground-truth-data")
print("Dataset base path:", path)

# Step 2 - Folder that contains fish_**
fish_image_path = os.path.join(path, "fish_image")

# Step 3 - Classes you want
selected_classes = ["fish_01", "fish_02", "fish_03", "fish_04", "fish_05", "fish_07"]

# Step 4 - Destination folder
new_dataset_path = "/content/fish_selected_6_classes.5"
os.makedirs(new_dataset_path, exist_ok=True)

# Create augmentation generator
aug_gen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True
)

TARGET_COUNT = 1000

# Step 5 - Process each class
for class_name in selected_classes:
    src = os.path.join(fish_image_path, class_name)
    dst = os.path.join(new_dataset_path, class_name)

    if not os.path.exists(src):
        print(f"Class not found: {class_name}")
        continue

    os.makedirs(dst, exist_ok=True)

    # Get all images in the source folder
    images = sorted(glob.glob(src + "/*"))

    # Copy first 2500 images (or all if less)
    copied_count = 0
    for img_path in images[:2500]:
        shutil.copy(img_path, dst)
        copied_count += 1

    print(f"Copied {copied_count} images from {class_name}")

    # If less than 1000 images → use augmentation
    if copied_count < TARGET_COUNT:
        needed = TARGET_COUNT - copied_count
        print(f"{class_name} needs augmentation: {needed} images")

        img_files = glob.glob(dst + "/*")  # existing (copied) images

        aug_count = 0
        idx = 0

        while aug_count < needed:
            img = load_img(img_files[idx])
            img_array = img_to_array(img)
            img_array = img_array.reshape((1,) + img_array.shape)

            # Generate 1 augmentation image at a time
            for batch in aug_gen.flow(img_array, batch_size=1,
                                      save_to_dir=dst,
                                      save_prefix="aug",
                                      save_format="jpg"):
                aug_count += 1
                break

            idx = (idx + 1) % len(img_files)

        print(f"Finished augmentation for {class_name}: now 1000 images total")

print("===== Dataset Processing Complete =====")

100%|██████████| 457M/457M [00:06<00:00, 78.2MB/s]

Extracting files...


Dataset base path: /root/.cache/kagglehub/datasets/madhushreesannigrahi/fish-recognition-ground-truth-data/versions/1
Copied 2500 images from fish_01
Copied 2500 images from fish_02
Copied 2500 images from fish_03
Copied 2500 images from fish_04
Copied 2500 images from fish_05
Copied 450 images from fish_07
fish_07 needs augmentation: 550 images
Finished augmentation for fish_07: now 1000 images total
===== Dataset Processing Complete =====


In [3]:
from google.colab import drive
drive.mount('/content/drive')

save_dir = "/content/drive/MyDrive/fish_results.6c"      # where results will be saved
import os
os.makedirs(save_dir, exist_ok=True)

Mounted at /content/drive


In [4]:
import numpy as np

img_paths = []
labels = []

# Make sure paths are correct
classes = sorted(os.listdir(new_dataset_path))

# All image extensions you want to include
extensions = ["*.jpg", "*.jpeg", "*.png", "*.JPG", "*.JPEG", "*.PNG"]

for class_name in classes:
    class_folder = os.path.join(new_dataset_path, class_name)

    for ext in extensions:
        for img_file in glob.glob(os.path.join(class_folder, ext)):
            img_paths.append(img_file)
            labels.append(class_name)

img_paths = np.array(img_paths)
labels = np.array(labels)

print("Total images:", len(img_paths))
print("Classes:", classes)

Total images: 13485
Classes: ['fish_01', 'fish_02', 'fish_03', 'fish_04', 'fish_05', 'fish_07']


In [5]:
from sklearn.preprocessing import LabelEncoder

label_enc = LabelEncoder()
labels_encoded = label_enc.fit_transform(labels)

num_classes = len(label_enc.classes_)
print("Number of classes:", num_classes)

Number of classes: 6


In [ ]:
import os
import shutil
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# -------------------------------------------------------
# CONFIG
# -------------------------------------------------------
IMG_SIZE = (128,128)
BATCH_SIZE = 32
K = 2    # number of folds
save_dir = "/content/resnet_kfold_results"
os.makedirs(save_dir, exist_ok=True)

# -------------------------------------------------------
# LOAD DATASET STRUCTURE
# -------------------------------------------------------
all_images = []
class_names = os.listdir(new_dataset_path)

for cls in class_names:
    class_path = os.path.join(new_dataset_path, cls)
    if not os.path.isdir(class_path):
        continue
    for img in os.listdir(class_path):
        all_images.append((os.path.join(class_path, img), cls))

all_images = np.array(all_images)
print("Total images:", len(all_images), " | Classes:", class_names)

# -------------------------------------------------------
# BUILD RESNET-18
# -------------------------------------------------------
def conv_block(x, filters, stride=1):
    shortcut = x

    # First conv
    x = layers.Conv2D(filters, 3, strides=stride, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    # Second conv
    x = layers.Conv2D(filters, 3, strides=1, padding="same")(x)
    x = layers.BatchNormalization()(x)

    # Match dimensions
    if stride != 1:
        shortcut = layers.Conv2D(filters, 1, strides=stride)(shortcut)
        shortcut = layers.BatchNormalization()(shortcut)

    x = layers.Add()([x, shortcut])
    x = layers.ReLU()(x)
    return x

def build_resnet18(num_classes):
    inputs = layers.Input(shape=(128,128,3))

    x = layers.Conv2D(64, 7, strides=2, padding="same")(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

    # ResNet-18 blocks
    x = conv_block(x, 64)
    x = conv_block(x, 64)

    x = conv_block(x, 128, stride=2)
    x = conv_block(x, 128)

    x = conv_block(x, 256, stride=2)
    x = conv_block(x, 256)

    x = conv_block(x, 512, stride=2)
    x = conv_block(x, 512)

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(256, activation="relu")(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)

    model = models.Model(inputs, outputs)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-4),
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model

# -------------------------------------------------------
# K-FOLD TRAINING
# -------------------------------------------------------
kf = KFold(n_splits=K, shuffle=True, random_state=42)
fold_no = 1
metrics_list = []

for train_index, test_index in kf.split(all_images):

    print(f"\n========================")
    print(f"      FOLD {fold_no}")
    print(f"========================")

    fold_folder = os.path.join(save_dir, f"fold_{fold_no}")
    os.makedirs(fold_folder, exist_ok=True)

    train_files = all_images[train_index]
    test_files = all_images[test_index]

    temp_train = os.path.join(save_dir, f"temp_train_{fold_no}")
    temp_test = os.path.join(save_dir, f"temp_test_{fold_no}")
    os.makedirs(temp_train, exist_ok=True)
    os.makedirs(temp_test, exist_ok=True)

    for cls in class_names:
        os.makedirs(os.path.join(temp_train, cls), exist_ok=True)
        os.makedirs(os.path.join(temp_test, cls), exist_ok=True)

    for filepath, cls in train_files:
        shutil.copy(filepath, os.path.join(temp_train, cls))

    for filepath, cls in test_files:
        shutil.copy(filepath, os.path.join(temp_test, cls))

    # DATA AUGMENTATION
    train_gen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        zoom_range=0.2,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True
    )

    test_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

    train_data = train_gen.flow_from_directory(
        temp_train,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode="categorical"
    )

    test_data = test_gen.flow_from_directory(
        temp_test,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode="categorical",
        shuffle=False
    )

    # BUILD MODEL
    model = build_resnet18(len(class_names))

    # TRAIN
    history = model.fit(
        train_data,
        epochs=10,
        validation_data=test_data,
        verbose=1
    )
    history_df = pd.DataFrame(history.history)
    history_df.to_csv(os.path.join(fold_folder, "training_history.csv"), index=False)

    # Plot Accuracy Curve
    plt.figure(figsize=(8,5))
    plt.plot(history.history['accuracy'], label='Train Acc')
    plt.plot(history.history['val_accuracy'], label='Val Acc')
    plt.title(f"Accuracy Curve - Fold {fold_no}")
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.legend()
    plt.grid()
    plt.savefig(os.path.join(fold_folder, "accuracy_curve.png"))
    plt.close()

    # Plot Loss Curve
    plt.figure(figsize=(8,5))
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Val Loss')
    plt.title(f"Loss Curve - Fold {fold_no}")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()
    plt.grid()
    plt.savefig(os.path.join(fold_folder, "loss_curve.png"))
    plt.close()


    # PREDICT
    y_true = test_data.classes
    y_pred = np.argmax(model.predict(test_data), axis=1)
    labels = list(test_data.class_indices.keys())

    # METRICS
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, average="macro")
    rec = recall_score(y_true, y_pred, average="macro")
    f1 = f1_score(y_true, y_pred, average="macro")

    metrics_list.append([fold_no, acc, prec, rec, f1])

    # SAVE REPORT
    with open(os.path.join(fold_folder, "classification_report.txt"), "w") as f:
        f.write(classification_report(y_true, y_pred, target_names=labels))

    # CONFUSION MATRIX
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10,7))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=labels, yticklabels=labels)
    plt.title(f"Confusion Matrix - Fold {fold_no}")
    plt.savefig(os.path.join(fold_folder, "confusion_matrix.png"))
    plt.close()

    # SAVE MODEL
    model.save(os.path.join(fold_folder, "resnet18_model.h5"))

    # CLEAN
    shutil.rmtree(temp_train)
    shutil.rmtree(temp_test)

    fold_no += 1

# SUMMARY CSV
df = pd.DataFrame(metrics_list, columns=["Fold","Accuracy","Precision","Recall","F1"])
df.to_csv(os.path.join(save_dir, "summary_metrics.csv"), index=False)

print("\nCross-validation completed!")


Total images: 13485  | Classes: ['fish_01', 'fish_07', 'fish_04', 'fish_03', 'fish_05', 'fish_02']

      FOLD 1
Found 6742 images belonging to 6 classes.
Found 6743 images belonging to 6 classes.


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 86s 284ms/step - accuracy: 0.7038 - loss: 0.7996 - val_accuracy: 0.0872 - val_loss: 5.1623
Epoch 2/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 43s 203ms/step - accuracy: 0.9004 - loss: 0.2910 - val_accuracy: 0.7246 - val_loss: 0.7763
Epoch 3/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 46s 220ms/step - accuracy: 0.9348 - loss: 0.1871 - val_accuracy: 0.9076 - val_loss: 0.2524
Epoch 4/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 43s 202ms/step - accuracy: 0.9343 - loss: 0.1792 - val_accuracy: 0.9540 - val_loss: 0.1345
Epoch 5/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 42s 201ms/step - accuracy: 0.9520 - loss: 0.1390 - val_accuracy: 0.9600 - val_loss: 0.1110
Epoch 6/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 42s 201ms/step - accuracy: 0.9650 - loss: 0.1056 - val_accuracy: 0.9623 - val_loss: 0.1098
Epoch 7/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 42s 201ms/step - accuracy: 0.9626 - loss: 0.1081 - val_accuracy: 0.9576 - val_loss: 0.1261
Epoch 8/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 43s 203ms/step - accuracy: 0.9621 - loss: 0


      FOLD 2
Found 6743 images belonging to 6 classes.
Found 6742 images belonging to 6 classes.


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 80s 275ms/step - accuracy: 0.6969 - loss: 0.8017 - val_accuracy: 0.2345 - val_loss: 4.4761
Epoch 2/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 43s 202ms/step - accuracy: 0.9036 - loss: 0.2827 - val_accuracy: 0.6688 - val_loss: 1.0019
Epoch 3/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 46s 219ms/step - accuracy: 0.9402 - loss: 0.1797 - val_accuracy: 0.8437 - val_loss: 0.5243
Epoch 4/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 42s 200ms/step - accuracy: 0.9495 - loss: 0.1501 - val_accuracy: 0.9257 - val_loss: 0.2144
Epoch 5/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 43s 203ms/step - accuracy: 0.9607 - loss: 0.1096 - val_accuracy: 0.9643 - val_loss: 0.1030
Epoch 6/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 43s 205ms/step - accuracy: 0.9657 - loss: 0.0924 - val_accuracy: 0.9462 - val_loss: 0.1725
Epoch 7/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 44s 206ms/step - accuracy: 0.9698 - loss: 0.0983 - val_accuracy: 0.9821 - val_loss: 0.0603
Epoch 8/10
211/211 ━━━━━━━━━━━━━━━━━━━━ 43s 205ms/step - accuracy: 0.9689 - loss: 0


Cross-validation completed!
